# T1490 Inhibit System Recovery

-----------------------------------------------------------------------

## Technique Description

Adversaries may delete or remove built-in operating system data and turn off services designed to aid in the recovery of a corrupted system to prevent recovery.(Citation: Talos Olympic Destroyer 2018)(Citation: FireEye WannaCry 2017) This may deny access to available backups and recovery options.

Operating systems may contain features that can help fix corrupted systems, such as a backup catalog, volume shadow copies, and automatic repair features. Adversaries may disable or delete system recovery features to augment the effects of [Data Destruction](https://attack.mitre.org/techniques/T1485) and [Data Encrypted for Impact](https://attack.mitre.org/techniques/T1486).(Citation: Talos Olympic Destroyer 2018)(Citation: FireEye WannaCry 2017)

A number of native Windows utilities have been used by adversaries to disable or delete system recovery features:

* <code>vssadmin.exe</code> can be used to delete all volume shadow copies on a system - <code>vssadmin.exe delete shadows /all /quiet</code>
* [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) can be used to delete volume shadow copies - <code>wmic shadowcopy delete</code>
* <code>wbadmin.exe</code> can be used to delete the Windows Backup Catalog - <code>wbadmin.exe delete catalog -quiet</code>
* <code>bcdedit.exe</code> can be used to disable automatic Windows recovery features by modifying boot configuration data - <code>bcdedit.exe /set {default} bootstatuspolicy ignoreallfailures & bcdedit /set {default} recoveryenabled no</code>

## Technique Detection

Use process monitoring to monitor the execution and command line parameters of binaries involved in inhibiting system recovery, such as vssadmin, wbadmin, and bcdedit. The Windows event logs, ex. Event ID 524 indicating a system catalog was deleted, may contain entries associated with suspicious activity.

Monitor the status of services involved in system recovery. Monitor the registry for changes associated with system recovery features (ex: the creation of <code>HKEY_CURRENT_USER\Software\Policies\Microsoft\PreviousVersions\DisableLocalPage</code>).

-----------------------------------------------------------------------

### Tactics:

  *   Impact

### Platforms:

  * Windows

  * macOS

  * Linux

### Data Sources:

  * **Windows Registry:** Windows Registry Key Modification

  * **Command:** Command Execution

  * **Service:** Service Metadata

  * **Process:** Process Creation

  * **File:** File Deletion

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1490)

  * [Fireeye Wannacry 2017](https://www.fireeye.com/blog/threat-research/2017/05/wannacry-malware-profile.html), Berry, A., Homan, J., and Eitzman, R. (2017, May 23). WannaCry Malware Profile. Retrieved March 15, 2019.

  * [Talos Olympic Destroyer 2018](https://blog.talosintelligence.com/2018/02/olympic-destroyer.html), Mercer, W. and Rascagneres, P. (2018, February 12). Olympic Destroyer Takes Aim At Winter Olympics. Retrieved March 14, 2019.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 8 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Adversaries will delete or remove built-in operating system data and turn off services designed to aid in the recovery of a corrupted system to prevent recovery.

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

## Detection Blindspots

- Information Here

## Analytical References

  * https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1490/T1490.md
* [eventtracker.com](https://kb.eventtracker.com/evtpass/evtpages/EventId_524_Microsoft-Windows-Backup_61998.asp#:~:text=Event%20ID%20%2D%20524&text=The%20System%20Catalog%20has%20been%20deleted.&text=This%20event%20is%20logged%20when%20the%20System%20Catalog%20has%20been%20deleted.&text=If%20the%20global%20catalog%20has,created%20earlier%20to%20perform%20recoveries)

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Use process monitoring to monitor the execution and command line parameters of binaries involved in inhibiting system recovery, such as vssadmin, wbadmin, and bcdedit.

#### Analytic 1

  * **Information:** 'Ryuk ransomware can delete backup files recursively with:'

  * **Source:** 'Windows Audits, Sysmon'

  * **Tool:** 'Kibana'

  * **Notes:** 'ex: del /s /f /q c:\*.VHD c:\*.bac c:\*.bak c:\*.wbcat c:\*.bkf c:\Backup*.* c:\backup*.* c:\*.set c:\*.win c:\*.dsk'

  * **Query:** ```process.name : del.exe```

#### Analytic 2

  * **Information:** 'bcdedit.exe can be used to disable automatic Windows recovery features by modifying boot configuration data'

  * **Source:** 'Windows Audits, Sysmon'

  * **Tool:** 'Kibana'

  * **Notes:** 'ex: bcdedit.exe /set {{default}} bootstatuspolicy ignoreallfailures & bcdedit /set {{default}} recoveryenabled no'

  * **Query:** ```process.name : bcdedit.exe```

#### Analytic 3

  * **Information:** 'vssadmin.exe can be used to delete all volume shadow copies on a system
  
  Windows Management Instrumentation can be used to delete volume shadow copies
  wbadmin.exe can be used to delete the Windows Backup Catalog'

  * **Source:** 'Windows Audits, Sysmon'

  * **Tool:** 'Kibana'

  * **Notes:** 'ex: vssadmin.exe delete shadows /all /quiet
  ex:wmic shadowcopy delete
  ex: wbadmin.exe delete catalog -quiet'

  * **Query:** ```process.name : vssadmin.exe or wmic.exe or wbadmin.exe AND process.command_line : *delete*```

#### Analytic 4

  * **Information:** 'Monitor the status of services involved in system recovery. Monitor the registry for changes associated with system recovery features'

  * **Source:** 'Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Event.code : 12 or 13 or 14 AND TargetObject : HKEY_CURRENT_USER\Software\Policies\Microsoft\PreviousVersions\DisableLocalPage```
  
#### Analytic 5

  * **Information:** 'Event.code 4104'

  * **Source:** 'Sysmon'

  * **Tool:** 'Kibana'

  * **Notes:** 'ex: Get-WmiObject Win32_Shadowcopy | ForEach-Object {$_.Delete();}'

  * **Query:** ```Event.code 4104 AND message contains Win32_Shadowcopy```

#### Analytic 6

  * **Information:** '524 – The System Catalog has been deleted'

  * **Source:** 'Windows Audits'

  * **Tool:** 'Kibana'

  * **Notes:** 'Event Id is from Microsoft-Windows-Backup logs and might not be included in the general shipping of logs to kibana.'

  * **Query:** ```Event.code : 524```

-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```


